Double-check the filters proposed. 

In [25]:
import numpy as np
import pandas as pd
import psycopg2

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.style
matplotlib.style.use('ggplot')
%matplotlib inline

In [26]:
# create a database connection
sqluser = 'mimic'
dbname = 'mimic'
schema_name = 'mimiciii'

cur = None

In [111]:
if cur: 
    cur.close()
    con.close()

con = psycopg2.connect(dbname = dbname, user = sqluser)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

Get echos that can be associated to an icustay_id. 

In [73]:
query = \
"""
SELECT *
FROM echo_icustay
"""
echos = pd.read_sql_query(query, con)
echos.shape

(25512, 7)

In [72]:
echos.head()

,row_id,charttime,icustay_id,hadm_id,subject_id,intime,outtime
0,67775,2170-11-08 12:03:00,206613,106296,269,2170-11-05 11:05:29,2170-11-08 17:46:57
1,73508,2186-12-25 23:54:00,210407,164716,272,2186-12-25 21:08:04,2186-12-27 12:01:13
2,99418,2114-07-03 11:54:00,254851,130546,274,2114-06-28 22:28:44,2114-07-07 18:01:16
3,75764,2164-06-15 10:26:00,204407,192224,279,2164-06-14 21:18:27,2164-06-18 22:53:12
4,60894,2101-10-24 09:50:00,257572,111199,281,2101-10-18 04:45:22,2101-10-25 22:29:25


In [73]:
len(echos['icustay_id'].unique())

19752

In [74]:
len(echos['hadm_id'].unique())

19244

In [75]:
len(echos['subject_id'].unique())

17122

Compute variables required for filtering, namely
+ Time of echo with respect to icustay intime
+ Whether or not patient was on a vasopressor during the icustay
+ Whether or not patient has an excluded diagnosis during the hospital admission
+ Age on admission to the ICU

Check the two hardest. Firstly, on whether or not the patient was on a vasopressor during the icustay: Look up prescriptions, filter for prescriptions that are vasopressors, count the number of unique icustay_ids. 

In [80]:
query = \
"""
SELECT ps.* FROM prescriptions ps
INNER JOIN d_prescriptions_vaso dpv
ON (ps.drug IS NOT DISTINCT FROM dpv.drug
    AND ps.drug_name_poe IS NOT DISTINCT FROM dpv.drug_name_poe
    AND ps.drug_name_generic IS NOT DISTINCT FROM dpv.drug_name_generic
    AND ps.route IS NOT DISTINCT FROM dpv.route)
INNER JOIN echo_icustay ei
ON ps.icustay_id = ei.icustay_id
;
"""
ps = pd.read_sql_query(query, con)
ps.head()

,row_id,subject_id,hadm_id,icustay_id,startdate,enddate,drug_type,drug,drug_name_poe,drug_name_generic,formulary_drug_cd,gsn,ndc,prod_strength,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,route
0,2350267,25,129635,203487,2160-11-02,2160-11-03,MAIN,Norepinephrine,None,None,LEVO4I,004939,00074144304,4MG/4ML AMP,4,mg,1,AMP,IV DRIP
1,1379347,21,111970,216859,2135-01-31,2135-02-06,MAIN,Norepinephrine,None,None,LEVO4I,028633,00703115303,4mg/4mL Amp,8,mg,2,AMP,IV DRIP
2,1379366,21,111970,216859,2135-02-05,2135-02-06,MAIN,Vasopressin,None,None,VASO20I,006611,61570042001,20U/ML VIAL,100,UNIT,5,ml,IV DRIP
3,1381496,36,182104,280987,2131-05-04,2131-05-04,MAIN,Phenylephrine,None,None,PHEN12SYR,005066,00409180001,12MG/50ML SYR,12,mg,1,SYR,IV DRIP
4,2907879,35,166707,282039,2122-02-10,2122-02-11,MAIN,Phenylephrine,None,None,PHEN12SYR,005066,00409180001,12MG/50ML SYR,12,mg,1,SYR,IV DRIP


In [87]:
len(ps['icustay_id'].unique())

8425

In [74]:
query = \
"""
SELECT * FROM echo_filter_vars
"""
efv = pd.read_sql_query(query, con)
efv.head()

,row_id,charttime,icustay_id,hadm_id,subject_id,intime,outtime,intime_to_echo,echo_to_outtime,ps_vaso,diag_xc,age_at_intime,after_rowid,before_rowid,op_to_icu,icu_to_op,time_filter,age_filter
0,67775,2170-11-08 12:03:00,206613,106296,269,2170-11-05 11:05:29,2170-11-08 17:46:57,3 days 00:57:31,0 days 05:43:57,True,True,14635 days 11:05:29,NaN,NaN,NaT,NaT,False,True
1,73508,2186-12-25 23:54:00,210407,164716,272,2186-12-25 21:08:04,2186-12-27 12:01:13,0 days 02:45:56,1 days 12:07:13,False,True,24489 days 21:08:04,NaN,NaN,NaT,NaT,True,True
2,99418,2114-07-03 11:54:00,254851,130546,274,2114-06-28 22:28:44,2114-07-07 18:01:16,4 days 13:25:16,4 days 06:07:16,False,True,24130 days 22:28:44,NaN,NaN,NaT,NaT,False,True
3,75764,2164-06-15 10:26:00,204407,192224,279,2164-06-14 21:18:27,2164-06-18 22:53:12,0 days 13:07:33,3 days 12:27:12,True,False,27115 days 21:18:27,NaN,NaN,NaT,NaT,True,True
4,60894,2101-10-24 09:50:00,257572,111199,281,2101-10-18 04:45:22,2101-10-25 22:29:25,6 days 05:04:38,1 days 12:39:25,False,True,21906 days 04:45:22,NaN,NaN,NaT,NaT,False,True


Ok. The numbers agree now. This is the number of icustays associated with ps_vaso. 

In [75]:
len(efv.loc[efv['ps_vaso'], 'icustay_id'].unique())

8425

In [76]:
len(efv.loc[efv['ps_vaso'], 'hadm_id'].unique())

8318

In [77]:
len(efv.loc[efv['ps_vaso'], 'subject_id'].unique())

7897

Also, check whether or not the patient has an excluded diagnosis. To do that, get the list of diagnoses to be excluded, filter the list of diagnoses actually to be excluded. 

In [98]:
query = \
"""
SELECT * FROM d_diagnoses_xc_annot
"""
dxc = pd.read_sql_query(query, con)
dxc.head()

,icd9_code,short_title,long_title,k_card,k_heart,k_hemorrhag,k_bleed,k_embolism,k_shock,k_clot,num,exclude,comments
0,3641,Meningococc pericarditis,Meningococcal pericarditis,1,0,0,0,0,0,0,11,1,None
1,4082,Toxic shock syndrome,Toxic shock syndrome,0,0,0,0,0,1,0,25,0,None
2,521,Varicella pneumonitis,Varicella (hemorrhagic) pneumonitis,0,0,1,0,0,0,0,4,1,None
3,860,Chagas disease of heart,Chagas' disease with heart involvement,0,1,0,0,0,0,0,1,1,None
4,11281,Candidal endocarditis,Candidal endocarditis,1,0,0,0,0,0,0,6,1,None


In [99]:
dxc_icd9s = set(dxc.loc[dxc['exclude'] == 1, 'icd9_code'].unique())

In [100]:
query = \
"""
SELECT * FROM DIAGNOSES_ICD
"""
dg = pd.read_sql_query(query, con)
dg.head()

,row_id,subject_id,hadm_id,seq_num,icd9_code
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [101]:
dg_ = dg.loc[dg['icd9_code'].apply(lambda d: d in dxc_icd9s)]
dg_.head()

,row_id,subject_id,hadm_id,seq_num,icd9_code
4,1301,109,172335,5.0,4254
28,1488,112,174105,1.0,53100
29,1489,112,174105,2.0,41071
51,1511,115,114585,10.0,4280
67,1527,117,140784,5.0,4280


In [104]:
len(set(dg_['hadm_id'].unique()) & set(echos['hadm_id']))

14125

In [78]:
len(efv.loc[efv['diag_xc'], 'icustay_id'].unique())

14586

In [79]:
len(efv.loc[efv['diag_xc'], 'hadm_id'].unique())

14125

In [80]:
len(efv.loc[efv['diag_xc'], 'subject_id'].unique())

12587

Ok. I think it agrees. Summarize in spreadsheet with the various filters on. 

In [81]:
len(efv.loc[efv['ps_vaso'] & ~efv['diag_xc'], 'icustay_id'].unique())

2328

In [82]:
len(efv.loc[efv['ps_vaso'] & ~efv['diag_xc'], 'hadm_id'].unique())

2315

In [83]:
len(efv.loc[efv['ps_vaso'] & ~efv['diag_xc'], 'subject_id'].unique())

2277

Time filter

In [84]:
time_filter = (efv['intime_to_echo'] > pd.Timedelta('-8 hours')) & (efv['intime_to_echo'] < pd.Timedelta('48 hours'))
len(efv.loc[time_filter, 'icustay_id'].unique())

16467

In [85]:
len(efv.loc[time_filter, 'hadm_id'].unique())

16115

In [86]:
len(efv.loc[time_filter, 'subject_id'].unique())

14567

On pressors, no excluded diagnoses, and time filter

In [87]:
len(efv.loc[efv['ps_vaso'] & ~efv['diag_xc'] & time_filter, 'icustay_id'].unique())

1967

In [88]:
len(efv.loc[efv['ps_vaso'] & ~efv['diag_xc'] & time_filter, 'hadm_id'].unique())

1964

In [89]:
len(efv.loc[efv['ps_vaso'] & ~efv['diag_xc'] & time_filter, 'subject_id'].unique())

1944

Age

In [100]:
efv.shape

(25512, 18)

In [101]:
ef.shape

(6672, 19)

In [102]:
ef.head()

,row_id,charttime,icustay_id,hadm_id,subject_id,intime,outtime,intime_to_echo,echo_to_outtime,ps_vaso,diag_xc,age_at_intime,after_rowid,before_rowid,op_to_icu,icu_to_op,time_filter,age_filter,echo_seq
0,67326,2135-02-01 15:42:00,216859,111970,21,2135-01-30 20:53:34,2135-02-08 05:38:46,1 days 18:48:26,6 days 13:56:46,True,False,32051 days 20:53:34,NaN,NaN,NaT,NaT,True,True,1
1,78499,2160-11-02 11:16:00,203487,129635,25,2160-11-02 03:16:23,2160-11-05 16:23:27,0 days 07:59:37,3 days 05:07:27,True,False,21511 days 03:16:23,NaN,NaN,NaT,NaT,True,True,1
2,84686,2122-02-10 11:37:00,282039,166707,35,2122-02-10 09:39:59,2122-02-15 18:41:01,0 days 01:57:01,5 days 07:04:01,True,False,25344 days 09:39:59,NaN,NaN,NaT,NaT,True,True,1
3,67805,2131-05-04 10:41:00,280987,182104,36,2131-05-04 10:29:14,2131-05-05 13:07:03,0 days 00:11:46,1 days 02:26:03,True,False,25443 days 10:29:14,NaN,NaN,NaT,NaT,True,True,1
4,67118,2113-02-15 17:20:00,216609,116009,62,2113-02-15 00:20:44,2113-02-17 20:09:46,0 days 16:59:16,2 days 02:49:46,True,False,25097 days 00:20:44,NaN,NaN,NaT,NaT,True,True,1


In [103]:
ef.loc[(ef['age_at_intime'] > np.timedelta64('18', 'Y')) != ef['age_filter']].head()

,row_id,charttime,icustay_id,hadm_id,subject_id,intime,outtime,intime_to_echo,echo_to_outtime,ps_vaso,diag_xc,age_at_intime,after_rowid,before_rowid,op_to_icu,icu_to_op,time_filter,age_filter,echo_seq
16,105319,2184-01-22 08:47:00,252789,176764,175,2184-01-21 21:22:19,2184-01-31 13:45:57,0 days 11:24:41,9 days 04:58:57,True,False,-103922 days +21:47:45.290448,NaN,NaN,NaT,NaT,True,True,1
99,69981,2197-06-20 14:17:00,243195,156708,1121,2197-06-20 10:30:39,2197-06-22 17:08:02,0 days 03:46:21,2 days 02:51:02,True,False,-100498 days +10:56:05.290448,NaN,NaN,NaT,NaT,True,True,1
128,96511,2171-11-14 15:52:00,217900,113063,1581,2171-11-12 18:13:14,2171-11-15 20:21:14,1 days 21:38:46,1 days 04:29:14,True,False,-104004 days +18:38:40.290448,NaN,NaN,NaT,NaT,True,True,1
131,69043,2192-10-24 16:24:00,240468,102458,1621,2192-10-23 06:59:25,2192-10-28 19:49:39,1 days 09:24:35,4 days 03:25:39,True,False,-104004 days +07:24:51.290448,NaN,NaN,NaT,NaT,True,True,1
132,96437,2187-12-03 11:39:00,259602,174523,1641,2187-12-02 18:46:17,2187-12-05 16:17:30,0 days 16:52:43,2 days 04:38:30,True,False,-104004 days +19:11:43.290448,96436.0,NaN,221 days 02:59:00,NaT,True,True,1


In [285]:
query = \
"""
SELECT * FROM icustays ic
INNER JOIN patients pt
ON ic.subject_id = pt.subject_id
WHERE ic.subject_id = '20249'; 
"""
check = pd.read_sql_query(query, con)
check.head()

,row_id,subject_id,hadm_id,icustay_id,dbsource,first_careunit,last_careunit,first_wardid,last_wardid,intime,outtime,los,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,25703,20249,152504,206946,carevue,CCU,CCU,57,57,2146-10-04 16:05:22,2146-10-06 18:52:58,2.1164,19108,20249,M,1846-10-04,2149-02-09,2149-02-09,2149-02-09,1
1,25704,20249,175360,243283,carevue,CCU,CCU,7,7,2149-02-06 15:54:53,2149-02-10 07:33:42,3.6520,19108,20249,M,1846-10-04,2149-02-09,2149-02-09,2149-02-09,1


Ok. Age is really awkward because of the older patients whose age are computed as negative numbers, for some reason. It looks like sql deals with is correctly, but somehow my python code does not. 

In [96]:
age_filter = (ef['age_at_intime'] > np.timedelta64('18', 'Y')) | (ef['age_at_intime'] < np.timedelta64(0))

On pressors + no excluded diagnosis + time + age

In [97]:
len(efv.loc[efv['ps_vaso'] & ~efv['diag_xc'] & time_filter & age_filter, 'icustay_id'].unique())

318

In [98]:
len(efv.loc[efv['ps_vaso'] & ~efv['diag_xc'] & time_filter & age_filter, 'hadm_id'].unique())

318

In [99]:
len(efv.loc[efv['ps_vaso'] & ~efv['diag_xc'] & time_filter & age_filter, 'subject_id'].unique())

313

Break down the diagnosis exclusions. 

In [104]:
query = \
"""
SELECT ei.icustay_id, ei.hadm_id, ei.subject_id
    ,dx.icd9_code, dx.long_title
FROM d_diagnoses_xc_annot dx
INNER JOIN diagnoses_icd di
    ON di.icd9_code = dx.icd9_code
INNER JOIN echo_icustay ei
    ON di.hadm_id = ei.hadm_id -- join on hospital admission
WHERE dx.exclude = 1
"""
dxc = pd.read_sql_query(query, con)
dxc.head()

,icustay_id,hadm_id,subject_id,icd9_code,long_title
0,262652,172335,109,4254,Other primary cardiomyopathies
1,232514,114585,115,4280,"Congestive heart failure, unspecified"
2,227264,195632,123,41011,Acute myocardial infarction of other anterior ...
3,225073,134369,124,9971,"Cardiac complications, not elsewhere classified"
4,256064,138376,124,42833,Acute on chronic diastolic heart failure


In [166]:
dxc_ = pd.concat([dxc[['icustay_id', 'icd9_code', 'long_title']].drop_duplicates().groupby(['icd9_code', 'long_title']).count(), 
           dxc[['hadm_id', 'icd9_code', 'long_title']].drop_duplicates().groupby(['icd9_code', 'long_title']).count(), 
           dxc[['subject_id', 'icd9_code', 'long_title']].drop_duplicates().groupby(['icd9_code', 'long_title']).count()], 
           axis = 1)
dxc_.to_csv("../../data/generated/D_DIAGNOSES_XC_COUNTS.csv")
dxc_.head()

,,icustay_id,hadm_id,subject_id
icd9_code,long_title,,,
11281,Candidal endocarditis,5,4,3
1510,Malignant neoplasm of cardia,14,14,14
1641,Malignant neoplasm of heart,1,1,1
2127,Benign neoplasm of heart,31,30,30
2851,Acute posthemorrhagic anemia,1482,1421,1401


In [165]:
dxc_.sort_values('icustay_id', ascending = False).head(n = 10)

,,icustay_id,hadm_id,subject_id
icd9_code,long_title,,,
4280,"Congestive heart failure, unspecified",7269,7021,6135
41071,"Subendocardial infarction, initial episode of care",2221,2123,2048
2851,Acute posthemorrhagic anemia,1482,1421,1401
9971,"Cardiac complications, not elsewhere classified",1472,1422,1409
42789,Other specified cardiac dysrhythmias,1135,1111,1089
4271,Paroxysmal ventricular tachycardia,1112,1069,1043
78551,Cardiogenic shock,1025,972,951
4275,Cardiac arrest,959,895,888
4254,Other primary cardiomyopathies,952,911,838


Perhaps count the number of excluded diagnosis assigned to each id. 

In [160]:
dxc[['icustay_id', 'icd9_code']].drop_duplicates().groupby('icustay_id').count()['icd9_code'].value_counts().sort_index()

1     5053
2     4365
3     2795
4     1409
5      599
6      215
7       91
8       43
9       14
10       1
11       1
Name: icd9_code, dtype: int64

In [162]:
dxc[['hadm_id', 'icd9_code']].drop_duplicates().groupby('hadm_id').count()['icd9_code'].value_counts().sort_index()

1     4972
2     4235
3     2691
4     1335
5      567
6      196
7       81
8       37
9        9
10       1
11       1
Name: icd9_code, dtype: int64

In [163]:
dxc[['subject_id', 'icd9_code']].drop_duplicates().groupby('subject_id').count()['icd9_code'].value_counts().sort_index()

1     4344
2     3532
3     2276
4     1244
5      630
6      286
7      139
8       74
9       38
10       8
11       8
12       5
13       3
Name: icd9_code, dtype: int64

Quick count

In [105]:
query = \
"""
SELECT * FROM echo_filtered; 
"""
ef = pd.read_sql_query(query, con)
ef.head()

,row_id,charttime,icustay_id,hadm_id,subject_id,intime,outtime,intime_to_echo,echo_to_outtime,ps_vaso,diag_xc,age_at_intime,after_rowid,before_rowid,op_to_icu,icu_to_op,time_filter,age_filter,echo_seq
0,67326,2135-02-01 15:42:00,216859,111970,21,2135-01-30 20:53:34,2135-02-08 05:38:46,1 days 18:48:26,6 days 13:56:46,True,False,32051 days 20:53:34,NaN,NaN,NaT,NaT,True,True,1
1,67805,2131-05-04 10:41:00,280987,182104,36,2131-05-04 10:29:14,2131-05-05 13:07:03,0 days 00:11:46,1 days 02:26:03,True,False,25443 days 10:29:14,NaN,NaN,NaT,NaT,True,True,1
2,67118,2113-02-15 17:20:00,216609,116009,62,2113-02-15 00:20:44,2113-02-17 20:09:46,0 days 16:59:16,2 days 02:49:46,True,False,25097 days 00:20:44,NaN,NaN,NaT,NaT,True,True,1
3,80320,2147-04-05 13:50:00,281607,112086,75,2147-04-05 10:35:50,2147-04-06 16:31:18,0 days 03:14:10,1 days 02:41:18,True,False,28018 days 10:35:50,NaN,NaN,NaT,NaT,True,True,1
4,80083,2146-04-06 14:05:00,206222,190243,86,2146-04-06 11:51:54,2146-04-07 10:27:02,0 days 02:13:06,0 days 20:22:02,True,False,16521 days 11:51:54,NaN,NaN,NaT,NaT,True,True,1


In [106]:
len(ef['icustay_id'].unique())

1931

In [107]:
len(ef['hadm_id'].unique())

1931

In [108]:
len(ef['subject_id'].unique())

1931

In [109]:
ef['subject_id'].value_counts().value_counts()

1    1930
2       1
Name: subject_id, dtype: int64

One weird datapoint: 

In [110]:
ef.loc[ef['subject_id'] == 26139]

,row_id,charttime,icustay_id,hadm_id,subject_id,intime,outtime,intime_to_echo,echo_to_outtime,ps_vaso,diag_xc,age_at_intime,after_rowid,before_rowid,op_to_icu,icu_to_op,time_filter,age_filter,echo_seq
563,69146,2132-10-31 11:26:00,298972,175355,26139,2132-10-30 01:28:20,2132-11-23 17:34:28,1 days 09:57:40,23 days 06:08:28,True,False,25951 days 01:28:20,NaN,NaN,NaT,NaT,True,True,1
564,69145,2132-10-31 11:26:00,298972,175355,26139,2132-10-30 01:28:20,2132-11-23 17:34:28,1 days 09:57:40,23 days 06:08:28,True,False,25951 days 01:28:20,NaN,NaN,NaT,NaT,True,True,1


Pulling the echo data, this is because there were two different echos, one which is Portable TTE (Focused views) Color Doppler only, and other which was Portable TTE (Complete) Full Doppler and color Doppler. 

Note that the echodata gives height, weight, bsa, bp bpsys, bpdias and hr. 